<a href="https://colab.research.google.com/github/Joshuas41/Sentiment-Analysis-With-LLM/blob/main/SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/SentimentAnalysisWithLLM
!ls

/content/drive/MyDrive/SentimentAnalysisWithLLM
amazon_alexa.tsv


In [ ]:
import pandas as pd

data = pd.read_csv('amazon_alexa.tsv', sep='\t')
data.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [ ]:
mydata = data[['verified_reviews','feedback']]
mydata.columns = ['review','label']

mydata.head()

,review,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
mydata.value_counts('label')

label
1    2893
0     257
Name: count, dtype: int64

In [ ]:
# Count the occurrences of each label
label_counts = mydata["label"].value_counts()

# Get the number of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

# Drop rows from the majority class randomly
if rows_to_drop > 0:
   data_majority = mydata[mydata["label"] == 1]
   data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
   data_balanced = mydata.copy()

# Check the new class balance
print(data_balanced["label"].value_counts())

label
1    257
0    257
Name: count, dtype: int64


In [ ]:
import pandas as pd
import re

# Extract the review column as a list
reviews = data_balanced['review'].tolist()

# Clean the text in the list
cleaned_reviews = [clean_text(str(review)) if not isinstance(review, str) else clean_text(review) for review in reviews] # Cast to string if not already a string

# Add the cleaned reviews as a new column to the DataFrame
data_balanced['clean_reviews'] = cleaned_reviews

def clean_text(text):
  # Remove special characters and punctuation
  text = re.sub(r"[^\w\s]", " ", text)

  # Remove single characters
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)

  # Remove HTML tags
  text = re.sub(r"<[^>]*>", " ", text)

  # Lowercase the text
  text = text.lower()

  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text)

  # Trim leading and trailing spaces
  text = text.strip()

  return text

In [ ]:
data_balanced

,review,label,clean_reviews
6,"Without having a cellphone, I cannot use many ...",1,without having cellphone cannot use many of he...
21,"We love Alexa! We use her to play music, play ...",1,we love alexa we use her to play music play ra...
27,Sounds great!! Love them!,1,sounds great love them
28,Fun item to play with and get used to using. ...,1,fun item to play with and get used to using so...
29,Just like the other one,1,just like the other one
...,...,...,...
3114,,1,
3122,I dislike that it confuses my requests all the...,1,dislike that it confuses my requests all the time
3128,Outstanding product - easy to use. works great,1,outstanding product easy to use works great
3135,I loved it does exactly what it says,1,loved it does exactly what it says


In [ ]:
import pandas as pd

# Assuming your DataFrame is called "df"
total_rows = len(data_balanced)
test_size = int(total_rows * 0.95)

# Randomly sample train_size rows for the training set
test_set = data_balanced.sample(test_size)

# Get the remaining rows for the test set
train_set = data_balanced.drop(test_set.index)

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 19.5 MB/s eta 0:00:00


In [ ]:
# Necessary packages
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
response = model.generate_content("What is the meaning of life?")

to_markdown(response.text)

CPU times: user 124 ms, sys: 13.4 ms, total: 138 ms
Wall time: 8.87 s


> The meaning of life is a deeply personal and subjective question that has been pondered by philosophers, theologians, and individuals throughout history. There is no universally accepted answer, and different people may find meaning in different aspects of their lives. Some common themes that emerge when discussing the meaning of life include:
> 
> 1. **Purpose and Fulfillment:** Many people find meaning in having a sense of purpose and working towards goals that they find fulfilling and meaningful. This could be related to a career, a creative pursuit, or making a difference in the world.
> 
> 2. **Relationships and Connections:** Strong and meaningful relationships with others can provide a sense of purpose and belonging. Nurturing these relationships and fostering a sense of community can contribute to a feeling of fulfillment.
> 
> 3. **Personal Growth and Development:** The journey of personal growth and self-discovery can be deeply meaningful. Embracing challenges, learning new things, and striving to become the best version of oneself can provide a sense of accomplishment and fulfillment.
> 
> 4. **Contribution to Society:** Making a positive contribution to the world, whether through acts of kindness, volunteering, or creating something that benefits others, can give life a sense of purpose and meaning.
> 
> 5. **Experiences and Moments:** Life is made up of countless moments and experiences. Some may be small and seemingly insignificant, while others can be profound and life-changing. Savoring these moments and fully engaging in the present can enhance our appreciation for life and give it meaning.
> 
> 6. **Legacy and Impact:** While we may not be here forever, the legacy we leave behind can have a lasting impact on the lives of others. Passing on our knowledge, values, and experiences can contribute to a sense of meaning and purpose.
> 
> Ultimately, the meaning of life is something that each individual must discover for themselves. It is an ongoing journey of exploration, reflection, and personal growth. There is no right or wrong answer, and the meaning of life can evolve and change over time as we gain new experiences and perspectives.

In [ ]:
test_set_sample = test_set.sample(20)

test_set_sample['pred_label'] = ''

test_set_sample

,review,label,clean_reviews,pred_label
2866,The volume is very low,0,the volume is very low,
2758,Easy to set up and connect with smart devices....,1,easy to set up and connect with smart devices ...,
1215,Yes it met my expectations. The snooze on the...,1,yes it met my expectations the snooze on the a...,
1643,Love it!!,1,love it,
2242,Perfect,1,perfect,
1278,Great product EXCEPT there is no possible way ...,0,great product except there is no possible way ...,
1146,I love it. I made a group with my echo dot so...,1,love it made group with my echo dot so get mus...,
299,Too difficult to set up. It keeps timing out ...,0,too difficult to set up it keeps timing out be...,
2716,The only negative we have on this product is t...,0,the only negative we have on this product is t...,
1298,"Sound quality is less than a dot, microphones ...",1,sound quality is less than dot microphones don...,


In [ ]:
# Convert the DataFrame to JSON using the to_json() method

json_data = test_set_sample[['clean_reviews','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"clean_reviews":"the volume is very low","pred_label":""},{"clean_reviews":"easy to set up and connect with smart devices just what needed","pred_label":""},{"clean_reviews":"yes it met my expectations the snooze on the alarm needs but of work tap it and it snoozes swipe up and it prompts you to dismiss but without my glasses can read it and often it just snoozes then one day must have dismissed tomorrow alarm because it never went off have it set for mon thu fri don need to get up early more apparent indicator of the alarm set would be nice my old alarm had red light","pred_label":""},{"clean_reviews":"love it","pred_label":""},{"clean_reviews":"perfect","pred_label":""},{"clean_reviews":"great product except there is no possible way for it to act as clock the displays the time permanently when in clock mode the screen rotates between the clock face and things to try screen intended on buying several units for my house but do not need screen in the house that is permanently trying t

In [ ]:
prompt = f"""
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
{json_data}
```
"""

print(prompt)


You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
[{"clean_reviews":"the volume is very low","pred_label":""},{"clean_reviews":"easy to set up and connect with smart devices just what needed","pred_label":""},{"clean_reviews":"yes it met my expectations the snooze on the alarm needs but of work tap it and it snoozes swipe up and it prompts you to dismiss but without my glasses can read it and often it just snoozes then one day must have dismissed tomorrow alarm because it never went off have it set for mon thu fri don need to get up early more apparent ind

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
[{"clean_reviews":"the volume is very low","pred_label":0},{"clean_reviews":"easy to set up and connect with smart devices just what needed","pred_label":1},{"clean_reviews":"yes it met my expectations the snooze on the alarm needs but of work tap it and it snoozes swipe up and it prompts you to dismiss but without my glasses can read it and often it just snoozes then one day must have dismissed tomorrow alarm because it never went off have it set for mon thu fri don need to get up early more apparent indicator of the alarm set would be nice my old alarm had red light","pred_label":0},{"clean_reviews":"love it","pred_label":1},{"clean_reviews":"perfect","pred_label":1},{"clean_reviews":"great product except there is no possible way for it to act as clock the displays the time permanently when in clock mode the screen rotates between the clock face and things to try screen intended on buying several units for my house but do not need screen in the house that is permanently trying to

In [ ]:
import json

# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,clean_reviews,pred_label
0,the volume is very low,0
1,easy to set up and connect with smart devices ...,1
2,yes it met my expectations the snooze on the a...,0
3,love it,1
4,perfect,1
5,great product except there is no possible way ...,0
6,love it made group with my echo dot so get mus...,1
7,too difficult to set up it keeps timing out be...,0
8,the only negative we have on this product is t...,0
9,sound quality is less than dot microphones don...,0


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_sample['pred_label'] = df_sample['pred_label'].values
test_set_sample

,review,label,clean_reviews,pred_label
2866,The volume is very low,0,the volume is very low,0
2758,Easy to set up and connect with smart devices....,1,easy to set up and connect with smart devices ...,1
1215,Yes it met my expectations. The snooze on the...,1,yes it met my expectations the snooze on the a...,0
1643,Love it!!,1,love it,1
2242,Perfect,1,perfect,1
1278,Great product EXCEPT there is no possible way ...,0,great product except there is no possible way ...,0
1146,I love it. I made a group with my echo dot so...,1,love it made group with my echo dot so get mus...,1
299,Too difficult to set up. It keeps timing out ...,0,too difficult to set up it keeps timing out be...,0
2716,The only negative we have on this product is t...,0,the only negative we have on this product is t...,0
1298,"Sound quality is less than a dot, microphones ...",1,sound quality is less than dot microphones don...,0


In [ ]:
# Plotting confusion matrix on the predictions

from sklearn.metrics import confusion_matrix

y_true = test_set_sample["label"]
y_pred = test_set_sample["pred_label"]

confusion_matrix(y_true, y_pred)

array([[9, 0],
       [3, 8]])

In [ ]:
from sklearn.metrics import accuracy_score
y_true = test_set_sample["label"]
y_pred = test_set_sample["pred_label"]
accuracy = accuracy_score(y_true, y_pred)
print(accuracy)

0.85
